# 1. Fetch Training Data from TRA website

In [1]:
import time
import urllib

import numpy as np

captcha_img_url = "http://railway1.hinet.net/ImageOut.jsp;jsessionid=VMt6QoesgHAFxK8c4dRSb8HtC2F8O5KGem8UwEOrwe464dVyR6Ks!-170005252"
filename = "{}.jpg".format(int(1000*time.time()))

urllib.urlretrieve(captcha_img_url, filename)

('1488527560773.jpg', <httplib.HTTPMessage instance at 0x10f7c4ef0>)

# 2. Image Preprocess

## 2a. RGB to W/B Mode

In [3]:
from PIL import Image, ImageFilter, ImageEnhance

In [4]:
filename = "1488173184806.jpg"

img = Image.open(filename)
width, height = img.size

img.show()

# transfer the color mode
img_l = img.convert("L")
img_l.show()

img_1 = img.convert("1")
img_1.show()

In [131]:
pixdata = img.load()

for y in xrange(height):
    for x in xrange(width):
        if pixdata[x, y][0] < 128 or pixdata[x, y][1] < 128 or pixdata[x, y][2] < 128:
            pixdata[x, y] = (0, 0, 0, 255)
        else:
            pixdata[x, y] = (255, 255, 255, 255)

## 2b. Locate the Positions of 'Number' from Image

In [132]:
csgraph = []
#pixdata = img.convert("1").load()
for y in xrange(height):
    line = []
    for x in xrange(width):
        v = 0 if pixdata[x, y][0] > 0 else 1   # 轉置為二維矩陣，尋找哪個區塊是 1 的叢聚
        line.append(v)

    csgraph.append(line)
    
for line in csgraph:
    print "".join(str(v) for v in line)

00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
00000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [126]:
import numpy as np

csgraph = np.array(csgraph)
g = csgraph.copy()

results = [[]]

def DFS(x, y):
    global g, results
        
    g[x, y] = 0
    for dx in range(-1, 2, 1):
        for dy in range(-1, 2, 1):
            pos_x, pos_y = x+dx, y+dy

            if pos_x > -1 and pos_y > -1 and pos_x < height and pos_y < width and g[pos_x, pos_y]:
                DFS(pos_x, pos_y)
                
                results[-1].append((pos_x, pos_y))
                
for pos_i in range(0, len(g)):
    for pos_j in range(0, len(g[pos_i])):
        if g[pos_i, pos_j]:
            DFS(pos_i, pos_j)
            
            if results[-1]:
                if len(results[-1]) < 32:
                    del results[-1]
                
                results.append([])
                            
del results[-1]

## 2c. Cut off the digits from image

In [156]:
print width, height

border = 4
cropped_images = []
for poss in results:    
    top, left = min([x[0] for x in poss]), min([x[1] for x in poss])
    bottom, right = max([x[0] for x in poss]), max([x[1] for x in poss])
        
    left, top = max(0, left-border), max(0, top-border)
    right, bottom = min(width, right+border), min(height, bottom+border)
                
    cropped_img = img.crop((left, top, right, bottom))
    cropped_images.append(cropped_img)
    
    cropped_img.show()

200 60


## 2d. Save image file with numpy format

In [159]:
for idx, cropped_img in enumerate(cropped_images):
    binary = []
    pixdata = cropped_img.convert("L").load()
    for y in xrange(cropped_img.size[1]):
        line = []
        for x in xrange(cropped_img.size[0]):
            line.append(pixdata[x, y])

        binary.append(line)
    binary = np.array(binary)
        
    cropped_img.show()
    cropped_img.save("{}.jpg".format(idx))    
    with open("{}.npy".format(idx), "wb") as out_file:
        np.save(out_file, binary)